In [ ]:
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import random, os
from config import settings

In [ ]:
from visual_generator import VisualGenerator
gen = VisualGenerator()

In [ ]:
import numpy as np
activation_root = settings['activation_root']
conv3 = np.load(activation_root + 'conv3.npy')
conv4 = np.load(activation_root + 'conv4.npy')
conv5 = np.load(activation_root + 'conv5.npy')

In [ ]:
u, s, v = np.linalg.svd(conv5, full_matrices=False)

In [ ]:
image_list_file = open(activation_root + 'input.txt')
image_list = []
while True:
    path = image_list_file.readline()
    if not path:
        break
    image_list.append(path.strip())

In [ ]:
node_list = []
for index in range(256):
    node_list.append({'layer': 'conv5', 'index': index})

In [ ]:
image_feature = u[:, 0:2]
unit_feature = v[0:2, :]

In [ ]:
assert conv5.shape[0] == len(image_list) and conv5.shape[1] == len(node_list)

In [ ]:
nodes = []
for node, i in zip(node_list, range(len(node_list))):
    nodes.append({'layer': node['layer'], 'index': node['index'],
                  'coord': [unit_feature[1, i], unit_feature[0, i]]})
    
images = []
random_list = random.sample([i for i in range(len(image_list))], 1000)
image_list_new = []
image_feature_new = np.ndarray((1000, 2), np.float)
for i in range(1000):
    image_list_new.append(image_list[random_list[i]])
    image_feature_new[i, :] = image_feature[random_list[i], :]
image_list = image_list_new
image_feature = image_feature_new
for image_path, i in zip(image_list, range(1000)):
    images.append({'path': image_path,
                   'coord': [image_feature[i, 1], image_feature[i, 0]]})

In [ ]:
plot_img = gen.visualize_collage_image(images)
fig = plt.figure(figsize=(20, 13))
plt.imshow(plot_img)
plot_img = gen.visualize_collage_node(nodes)
fig = plt.figure(figsize=(20, 13))
plt.imshow(plot_img)

In [ ]:
from tsne.tsne import TSNEVisualizer
tsne_visualizer = TSNEVisualizer()

In [ ]:
def visualization_callback(coord, iter):
    if iter % 20 == 0:
        disp_nodes = []
        for node, i in zip(node_list, range(len(node_list))):
            disp_nodes.append({'layer': node['layer'], 'index': node['index'],
                               'coord': [coord[i, 1], coord[i, 0]]})
        plot_img = gen.visualize_collage_node(disp_nodes)
        fig = plt.figure(figsize=(20, 13))
        plt.imshow(plot_img)
        plt.show()
        
def visualization_callback_image(coord, iter):
    if iter % 20 == 0:
        disp_images = []
        for image, i in zip(image_list, range(len(image_list))):
            disp_images.append({'path': image,
                                'coord': [coord[i, 1], coord[i, 0]]})
        plot_img = gen.visualize_collage_image(disp_images)
        fig = plt.figure(figsize=(20, 13))
        plt.imshow(plot_img)
        plt.show()

In [ ]:
tsne_visualizer.tsne(unit_feature.transpose(), 2, 50, 10, visualization_callback)

In [ ]:
tsne_visualizer.tsne(image_feature, 2, 50, 10, visualization_callback_image)